In [1]:
# Import required libraries
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
# get Launch Site names
orte = list(spacex_df["Launch Site"].unique())
orte

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                     {'label': 'All Sites', 'value': 'ALL'},
                                                     {'label': orte[0], 'value': orte[0]},
                                                     {'label': orte[1], 'value': orte[1]},
                                                     {'label': orte[2], 'value': orte[2]},
                                                     {'label': orte[3], 'value': orte[3]},                                                
                                                     ],
                                             value='ALL',
                                             placeholder="Select a Launch Site here",
                                             searchable=True
                                             ),
                                html.Br(),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.P("Pie Chart:"),
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000, 
                                                marks={0:'0', 100:'100', 250:'250', 500:'500'},
                                                value=[min_payload, max_payload]),
                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(enteredSite):
    filtered_df = spacex_df
    if enteredSite == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == enteredSite] 
        filtered_df = filtered_df.groupby(["Launch Site","class"]).size().reset_index(name="class count")
        fig = px.pie(filtered_df, values='class count', 
        names='class', 
        title=f'Total Success Launches By Site {enteredSite}')
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])

def get_scatter_plot(enteredSite,MinMax):
    filtered_df = spacex_df[(spacex_df["Payload Mass (kg)"] >= MinMax[0]) & (spacex_df["Payload Mass (kg)"] <= MinMax[1])]
    if enteredSite == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y ="class",
        color='Booster Version Category',
        title="Payload Mass vs. Success")
        return fig
    else:
        # return the outcomes for a selected site
        filtered_df = filtered_df[spacex_df['Launch Site'] == enteredSite] 
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y ="class",
        color='Booster Version Category',
        title=f'Payload Mass vs. Success By Site {enteredSite}')
        return fig



In [2]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [13/Feb/2023 10:34:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 10:34:11] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 10:34:11] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 10:34:11] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Feb/2023 10:34:11] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Feb/2023 10:34:11] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Feb/2023 10:34:11] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Feb/2023 10:34:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 10:34:12] "POST /_dash-update-component HTTP/1.1" 200 -


In [ ]:
# Quit the app
if __name__ == '__main__':
    app.quit